# Use intake + STAC to facilitate loading landsat8 data into xarray

#### STAC 0.6 has a lot of improvements over 0.5, so let's try to use that. 

#### STAC 0.6 search api is not quite finished (for dynamic catalog generation)! so for now, just use static catalog
    
    * https://github.com/radiantearth/stac-spec 
    * https://github.com/sat-utils/sat-api 

DevelopmentSeed has put together a complete STAC catalog for landsat on AWS:

    * https://github.com/sat-utils/sat-stac-landsat 

The catalog is essentially a bunch of nested JSON files starting here:

    * https://landsat-stac.s3.amazonaws.com/catalog.json

You can traverse the 'child' links to get to a specific image, for example:
collection level metadata here:

    * https://landsat-stac.s3.amazonaws.com/landsat-8-l1/catalog.json

Listing of all images for a particular path/row:

    * https://landsat-stac.s3.amazonaws.com/landsat-8-l1/047/027/catalog.json

Metadata for a single image ("STAC item"):

    * https://landsat-stac.s3.amazonaws.com/landsat-8-l1/047/027/2018-01-13/LC80470272018013LGN00.json




In [ ]:
import intake
import hvplot.xarray
import requests
import xarray as xr
import pandas as pd
import os
import rasterio
import matplotlib.pyplot as plt
%matplotlib inline

## Get file paths from STAC 
* consider adding this functionality to a new intake-stac plugin? might want to incoporate these 
* STAC is geojson, so can convert directly to python dictionary and therefore pandas dataframe
* for satellite imagery stac items, they have polygon footprints with varying amounts of metadata, so well suited to a geopandas geodataframe
    * items can have many imagery assets (multiband images) having different resolutions. xarray best suited to loading images with same resolution and footprint

In [ ]:
def get_meta(stac_item):
    ''' turn STAC item into python dictionary'''
    #print(f'retrieving metadata for {stac_item}')
    stac_item = stac_item
    response = requests.get(stac_item)
    meta = response.json() #all metadata, could sort for tier1 here, best to use sat-api/sat-search
    return meta

In [ ]:
def get_links(stac_item):
    ''' dataframe of asset links from stac item'''
    response = requests.get(stac_item)
    meta = response.json()
    df = pd.DataFrame.from_dict(meta['assets'],orient='index')
    return df

In [ ]:
def create_dataframe(stac_catalog):
    ''' turn STAC catalog containing items into pandas dictionary '''
    dictionary = {} 
    response = requests.get(stac_catalog)
    inventory = response.json()
    items = [x['href'] for x in inventory['links'] if 'item' in x.values()]
    for i in items:
        url = os.path.join(os.path.dirname(stac_catalog),i)
        #print(url)
        meta = get_meta(url)
        #meta['properties']['index'] = meta['assets']['index']['href']
        meta['properties']['stac'] = url
        
        date = pd.to_datetime(meta['properties']['datetime'])
        dictionary[date] = meta['properties']
    
    df = pd.DataFrame.from_dict(dictionary, orient='index')
    df['datetime'] = pd.to_datetime(df.datetime.str[:10]) #for convenience later...
    return df

In [ ]:
# slow because using network to get geojson for each STAC item...
# could read info in parallel / avoid for loop?
baseurl = 'https://landsat-stac.s3.amazonaws.com/landsat-8-l1'
path = 47
row = 27
stac_subcat = f'{baseurl}/{path:03d}/{row:03d}/catalog.json'

DF = create_dataframe(stac_subcat)

In [ ]:
print(len(DF), 'STAC items in catalog:' ) 
DF.head()

In [ ]:
DF['landsat:tier'].unique()

In [ ]:
# get only T1 data
df = DF[DF['landsat:tier'] == 'T1'].sort_index()

In [ ]:
print(len(df), 'STAC items selected:' )
df.head()

In [ ]:
df.stac.iloc[0]

In [ ]:
get_links(df.stac.iloc[0])

# Single Item: All 30m bands from specific scene with intake catalog

* intake is helpful for putting the logic to load STAC items into an xarray DataArray behind the scenes
* would be nice to allow loading into a dataset instead of dataarray

In [ ]:
# inspired by: https://www.anaconda.com/blog/developer-blog/intake-parsing-data-from-filenames-and-paths/
# NOTE: incorrect dimensions / chunksize
# NOTE: bands have different resolutions / dimensions: https://www.usgs.gov/media/images/landsat-8-band-designations
cat = intake.Catalog('intake-landsat-30m.yml')
ds = cat.aws_landsat_8().to_dask()
ds

In [ ]:
# Slow... pulls 9 probably pulling 9 images to local memory? - would be better to pull high resolution overviews first for speed?
# Could create another intake source for just RGB thumbnails and display those...
#cat.aws_landsat_8.plot.band_image()

# Multiple items: Load all Tier1 data from specific row and column into an xarray Dataset

* need some new functionality in intake-xarray to do this 
* code below is made in hase, probable not very efficient
* first, browse all the pre-made RGB thumbnail images
* seems to be a lot happening with boto getting AWS credentials for every image

In [ ]:
#First... a thumbnail browser! NOTE: these thumbnail JPGs are not georeferenced... but RGB COG
# would be neat b/c we'd have coordinates and resolution updated w/ datashader
images = []
for i,row in df.iterrows():
    #print(row.datetime)
    da = cat.aws_landsat_8_thumbnails(path=int(row['eo:column']), 
                                     row=int(row['eo:row']), 
                                     product_id=row['landsat:product_id']).read()
    da = da.expand_dims('time')
    da = da.assign_coords(time=[row.datetime])
    images.append(da)

ds = xr.concat(images, 'time').to_dataset(name='z')

# NOTE: hvplot rgb only seems to work if 'band coordinate comes first!
coords = dict(band=(['band'], ds.coords['band'].values),
              time=(['time'], ds.coords['time'].values),
              y=(['y'], ds.coords['y'].values),
              x=(['x'], ds.coords['x'].values),
              )

ds = xr.Dataset({'z': (['time','band','y','x'], ds['z'].data.astype('uint8'))}, coords=coords)

In [ ]:
ds.hvplot.rgb('x','y', z='z',bands='band',groupby='time', width=700, height=500).options(invert_yaxis=True)

In [ ]:
def create_dataset(stac_catalog, template='intake-landsat-30m.yml'):
    ''' create an xarray.DataSet from tier1 data in landsat STAC catalog'''
    datasets = []
    # NOTE: slow b/c reading over network
    DF = create_dataframe(stac_catalog) #NOTE: alternatively use devseed sat-search first to filter larger catalog...
    df = DF[DF['landsat:tier'] == 'T1'].sort_index()
    df.index.name = 'time'
    
    cat = intake.Catalog('intake-landsat-30m.yml')
    #slow
    for i,row in df.iterrows():
        print(row.datetime)
        da = cat.aws_landsat_8(path=int(row['eo:column']), row=int(row['eo:row']), product_id=row['landsat:product_id']).to_dask()
        ds = da.to_dataset(dim='band')
        datasets.append(ds)
    
    DS = xr.concat(datasets, dim=df.index)
    
    return DS

In [ ]:
# NOTE: try rasterio context manager to avoid trying to get aws credentials w/ boto?
# How to speed this up?
stac_catalog = 'https://landsat-stac.s3.amazonaws.com/landsat-8-l1/047/027/catalog.json'
DS = create_dataset(stac_catalog)

In [ ]:
DS

# NDVI calculation

In [ ]:
# rechunk since chunk sizes is adjusted when loading scenes w/ different dimensions and do an NDVI calculation
ds = DS.chunk(dict(time=1, x=DS.dims['x'], y=512))
NDVI = (ds[5] - ds[4]) / (ds[5] + ds[4])
NDVI

In [ ]:
# Compute and visualize! - best to use dask for this...
#img = NDVI.hvplot('x', 'y', groupby='time', dynamic=True, rasterize=True, width=700, height=500, cmap='magma')
#img

# What if we want to work with images not in the same Path an Row?

### The answer is more difficult than you might expect!

* Landsat scenes are referenced on a global path/row grid system and overlap.
* Scenes are stored in different projected coordinate reference systems (UTM zones)
* because of the 'swath' nature of satellite imagery, we end up having to store a lot of nan values in our xarray dataset (maybe sparse arrays can help here?)
    * Eventually we want some intelligent way to access "mosaiic" and "composite" images (https://developers.google.com/earth-engine/ic_composite_mosaic)


Here is a simple example for a particulare date 2013-07-26 from path=47, row=27 and scene to south (row=28)
* I end up defering to GDAL, but could use rasterio python library to avoid external function calls

In [ ]:
north = df[df['datetime'] == "2013-07-26"].stac.values[0]

In [ ]:
img_north = get_links(north)['href']['B1']

In [ ]:
def get_l8item(stac_catalog, date):
    ''' only for landsat'''
    response = requests.get(stac_catalog)
    data = response.json()
    items = [x['href'] for x in data['links'] if 'item' in x.values()]
    item = [x for x in items if date in x]
    return item

In [ ]:
date = '2013-07-26'
stac_catalog = 'https://landsat-stac.s3.amazonaws.com/landsat-8-l1/047/028/catalog.json'
get_l8item(stac_catalog, date)
# NOTE: two returned b/c tier1, tier2, etc

south = stac_catalog.replace('catalog.json','2013-07-26/LC80470282013207LGN01.json')
img_south = get_links(south)['href']['B1']

In [ ]:
# Create a VRT that merges these adjacent, slightly overlapping scenes
# https://www.gdal.org/gdalbuildvrt.html
# pixels from last file in list are used for overlaps
#cmd = f'gdal_merge.py -of VRT -n 0 -o merged.vrt /vsicurl/{img_north} /vsicurl/{img_south}' #won't write to vrt
cmd = f'gdalbuildvrt -overwrite -vrtnodata 0 -srcnodata 0 merged.vrt /vsicurl/{img_north} /vsicurl/{img_south}'

print(cmd)
!{cmd}

In [ ]:
# open this file with xarray
da = xr.open_rasterio('merged.vrt')

In [ ]:
da

In [ ]:
img = da.hvplot.image(rasterize=True, dynamic=True, datashade=True, width=700, height=500, cmap='magma')
img

# How about all of washington state?

In [ ]:
def query_sat_api(data, stac_version=0.5):
    '''https://github.com/sat-utils/sat-api'''
    
    if stac_version == 0.5:
        baseurl = 'https://sat-api.developmentseed.org/search/stac' #0.5
    elif stac_version == 0.6:
        baseurl = 'https://sat-api-dev.developmentseed.org/stac/search' #0.6

    r = requests.get(baseurl, params=data, timeout=100)
    #print(r.url)
    # Save Directly to dataframe
    # df = pd.DataFrame(r.json()[0])
    print('Saved results to response.json')
    with open('response.json', 'w') as j:
        j.write(r.text)

In [ ]:
# need to simplify multipolygon (w/ islands)
#with open('washington.json') as f:
#    aoi = f.read()

with open('wa-bbox.geojson') as f:
    aoi = f.read()

    
#All WA state in 2017
stac5 = {'c:id':'landsat-8-l1',
        'intersects':aoi,
        'datetime':'2017-07-01/2017-08-01',
        #'eo:cloud_cover':'0/50',
        'limit':1000}



In [ ]:
query_sat_api(stac5)

In [ ]:
# Load geojson w/ geopandas
import geopandas as gpd
gf = gpd.read_file('response.json')
gf = gf.sort_values('datetime')
print('records:', len(gf))
gf.head()

In [ ]:
# Filter tier 1 scenes
gf = gf[gf.id.str.endswith('T1')]
gf['datetime'] = pd.to_datetime(gf.datetime)
gf['date'] = gf.datetime.apply(lambda x: x.date())
print('tier1 scenes:', len(gf))

In [ ]:
# NOTE: landsat8 has a 16 day revisit time. So let's plot before the 18th and 
# after the 18th, should also make sure sorted by acquisition time!
thresh = pd.Timestamp('2017-07-17')
gf1 = gf[gf.datetime <= thresh]
gf2 = gf[gf.datetime > thresh]

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(8,4))
gf1.plot(column='date', edgecolor='k', cmap='magma', alpha=0.5, legend=True, ax=ax1)
gf2.plot(column='date', edgecolor='k', cmap='magma', alpha=0.5, legend=True, ax=ax2)
leg = ax1.get_legend()
leg.set_bbox_to_anchor((0.5, -0.5, 0.2, 0.2))
leg = ax2.get_legend()
leg.set_bbox_to_anchor((0.5, -0.5, 0.2, 0.2))

In [ ]:
# Psuedocode (this is where we want to get to)

''' 
cat = intake.Catalog('intake-landsat-stac.yml')

stac_params = {'c:id':'landsat-8-l1',
          'intersects':aoi,
          'datetime':'2017-07/2017-08'}

# select specific bands, resolution, coordinate system, resampling algorithm??
#gdal_params = {bands=[4,5],
#               res=10}

ds = cat.landsat_mosaic(**stac_params).to_dask()
'''

In [ ]:
def get_sat_api_item(pid, stac_version=0.5, band='B1'):
    '''https://github.com/sat-utils/sat-api'''
    
    data = {'c:id':'landsat-8-l1',
    'id':pid}
    
    if stac_version == 0.5:
        baseurl = 'https://sat-api.developmentseed.org/search/stac' #0.5
    elif stac_version == 0.6:
        baseurl = 'https://sat-api-dev.developmentseed.org/stac/search' #0.6

    r = requests.get(baseurl, params=data, timeout=100)
    #return r.json() returns a python dictionary
    item = r.json()
    
    url = item['features'][0]['assets'][band]['href']
    return url



In [ ]:
get_sat_api_item('LC08_L1TP_042027_20170702_20170715_01_T1', stac_version=0.5)

In [ ]:
# get links to all files that will be mosaiced
series = gf1.id.apply(get_sat_api_item) #slow!

In [ ]:
series.to_csv('my_list.txt', index=False, sep='\n')

In [ ]:
# Since frames span multiple UTM zones, need to put into common CRS (WGS84 lat/lon)
def utm2latlon(url):
    outvrt = os.path.basename(url).replace('TIF','vrt')
    cmd = f'gdalwarp -dstnodata 0 -t_srs EPSG:4326 -of VRT /vsicurl/{url} {outvrt}'
    #print(cmd)
    os.system(cmd)

In [ ]:
results = series.apply(utm2latlon)

In [ ]:
!gdalbuildvrt -overwrite -vrtnodata 0 -srcnodata 0 mosaic.vrt LC08*.vrt

In [ ]:
# open this file with xarray
da = xr.open_rasterio('mosaic.vrt', chunks=dict(band=1, y=2048, x=2048))

In [ ]:
da

In [ ]:
# IT WORKS!.... but....

# Taking way too long, not sure how much data is being pulled in the background

# See:
# https://github.com/pyviz/geoviews/issues/230#issuecomment-435379550

img = da.hvplot.image(rasterize=True, dynamic=True, datashade=True, width=700, height=500, cmap='magma')
img